# Import

In [1]:
import pandas as pd
import re
import datetime

In [2]:
data = pd.read_csv("/Users/niccdias/Desktop/IHS_Data.csv", header = 0)
data = data.drop(data.index[[0]])

In [3]:
leo_1 = pd.read_csv("~/Desktop/leo_3-3(1).csv", header = 0)
leo_2 = pd.read_csv("~/Desktop/leo_3-3(2).csv", header = 0)
leo_1 = leo_1.drop(leo_1.index[[0]])
leo_2 = leo_2.drop(leo_2.index[[0]])

#data2 = pd.read_csv("~/Desktop/nk_news.csv", header = 0)
#data2 = data2.drop(data2.index[[0]])

In [4]:
data2 = leo_1.append(leo_2)
data2 = data2.reset_index()
del data2['index']

# Define cleaning and formatting functions

In [5]:
# Create zero padding for day of the month
def zero_padding(date):
    match = re.search('([A-Za-z]+\s)(\d*)(,\s\d\d\d\d)', date)
    if match and len(match.groups(0)[1]) == 1:
        day_of_month = "0" + match.groups(0)[1]
        return match.groups(0)[0] + day_of_month + match.groups(0)[2]
    else:
        return date

In [6]:
# Convert to dates to datetime format
def ihs_datetimer(text):
    match = re.search('([A-Za-z]+\s)(\d\d)(,\s\d\d\d\d)', text)
    match2 = re.search('([A-Za-z]+\s\d\d\d\d)', text)
    if text == '0000':
        return 'ERROR'
    elif len(text) == 4:
        return pd.to_datetime(text, format='%Y')
    elif match:
        return pd.to_datetime(text, format='%B %d, %Y')
    elif match2:
        return pd.to_datetime(text, format='%B %Y')
    else:
        return 'ERROR'

# Define functions for building the dataframe

In [7]:
def doc_comp_co(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "COMPANY                       EFFECTIVE DATE           COMPANY STATUS\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[29:55].strip() != "":
            node = line[0:30].strip()
            if node == "":
                node = "Unknown"
            date = line[29:55].strip()
            ship_dict = {}
            ship_dict["relation"] = 'doc_comp_co'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:30].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

In [8]:
def flag_history(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "FLAG                          FLAG CODE                EFFECTIVE DATE\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[29:55].strip() != "":
            node = line[0:30].strip()
            if node == "":
                node = "Unknown"
            date = line[54:].strip()
            ship_dict = {}
            ship_dict["relation"] = 'flag'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:30].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

In [9]:
def name_history(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "VESSEL NAME                          EFFECTIVE DATE\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[37:].strip() != "":
            node = line[0:37].strip()
            if node == "":
                node = "Unknown"
            date = line[37:].strip()
            ship_dict = {}
            ship_dict["relation"] = 'ship_name'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:37].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

In [10]:
def operator_history(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "OPERATOR                      EFFECTIVE DATE           COMPANY STATUS\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[29:55].strip() != "":
            node = line[0:30].strip()
            if node == "":
                node = "Unknown"
            date = line[29:55].strip()
            ship_dict = {}
            ship_dict["relation"] = 'operator'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:30].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

In [11]:
def gbo_history(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "GROUP BENEFICIAL OWNER        EFFECTIVE DATE           COMPANY STATUS\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[29:55].strip() != "":
            node = line[0:30].strip()
            if node == "":
                node = "Unknown"
            date = line[29:55].strip()
            ship_dict = {}
            ship_dict["relation"] = 'gbo'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:30].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

In [12]:
def reg_owner_history(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "OWNER                         EFFECTIVE DATE           COMPANY STATUS\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[29:55].strip() != "":
            node = line[0:30].strip()
            if node == "":
                node = "Unknown"
            date = line[29:55].strip()
            ship_dict = {}
            ship_dict["relation"] = 'reg_owner'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:30].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

In [13]:
def ship_man_history(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "SHIP MANAGER                  EFFECTIVE DATE           COMPANY STATUS\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[29:55].strip() != "":
            node = line[0:30].strip()
            if node == "":
                node = "Unknown"
            date = line[29:55].strip()
            ship_dict = {}
            ship_dict["relation"] = 'ship_manager'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:30].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

In [14]:
def tech_man_history(text):
    today = pd.to_datetime(datetime.date.today())
    vessel = re.findall('IMO\) NUMBER: (\d*)', text)
    status = re.findall('SHIP STATUS:\s+(.*)\r\n\s*.*?[A-Z]+:*', text)
    header = "TECHNICAL MANAGER             EFFECTIVE DATE           COMPANY STATUS\r\n"
    header_index = text.find(header)
    table = text[header_index + len(header):]
    blank_index = table.find("\r\n\r\n")
    table = table[0:blank_index]
    lines = table.split('\r\n')
    for line in lines:
        if line[29:55].strip() != "":
            node = line[0:30].strip()
            if node == "":
                node = "Unknown"
            date = line[29:55].strip()
            ship_dict = {}
            ship_dict["relation"] = 'tech_manager'
            ship_dict["imo"] = pd.to_numeric(vessel[0])
            ship_dict["ship_current_status"] = status[0]
            ship_dict["node"] = node
            ship_dict["start_date"] = ihs_datetimer(zero_padding(date))
            ship_dict['end_date'] = today
            master_list.append(ship_dict)
            today = ihs_datetimer(zero_padding(date))
        else:
            rest_of_name = line[0:30].strip()
            master_list[-1]['node'] = master_list[-1]['node'] + " " + rest_of_name

# Build the dataframe and clean the flag names

In [15]:
master_list = []
data["column1"].apply(doc_comp_co)
data["column1"].apply(flag_history)
data["column1"].apply(name_history)
data["column1"].apply(operator_history)
data["column1"].apply(gbo_history)
data["column1"].apply(reg_owner_history)
data["column1"].apply(ship_man_history)
data["column1"].apply(tech_man_history)
df = pd.DataFrame(master_list)
df = df[df['end_date'] != 'ERROR']
df = df[df['start_date'] != 'ERROR']

In [15]:
master_list = []
data2["column1"].apply(doc_comp_co)
data2["column1"].apply(flag_history)
data2["column1"].apply(name_history)
data2["column1"].apply(operator_history)
data2["column1"].apply(gbo_history)
data2["column1"].apply(reg_owner_history)
data2["column1"].apply(ship_man_history)
data2["column1"].apply(tech_man_history)
df2 = pd.DataFrame(master_list)

# Find those baddies

In [16]:
def date_range(start, end):
    delta = end - start
    dates = []
    for i in range(delta.days + 1):
        dates.append(start + datetime.timedelta(days=i))
    return dates

In [17]:
def find_2270_baddies(imo):
    today = pd.to_datetime(datetime.date.today())
    global baddy_list
    global res_date
    global imo_list
    global vio_list
    if imo not in imo_list:
        vessel = df[df['imo'] == imo]
        nk_flag_periods = vessel[(vessel['relation'] == 'flag') & (vessel['node'] == "North Korea")]
        for start, end in zip(nk_flag_periods['start_date'], nk_flag_periods['end_date']):
            for index, row in vessel[vessel['relation'].isin(vio_list)].iterrows():
                nk_range = date_range(start, end)
                row_range = date_range(row['start_date'], row['end_date'])            
                overlap = set(nk_range).intersection(row_range)
                if overlap:
                    overlap_start = min(overlap)
                    overlap_end = max(overlap)
                    if (overlap_start <= today)  and  (overlap_end >= res_date):
                        baddy_dict = {}
                        baddy_dict['relation'] = row['relation']
                        baddy_dict['imo'] = row['imo']
                        baddy_dict['node'] = row['node']
                        baddy_dict['ship_current_status'] = row['ship_current_status']
                        baddy_list.append(baddy_dict)
        imo_list.append(imo)

In [20]:
current_nk_flags = df[(df['node'] == "North Korea") & (df['end_date'] == pd.to_datetime(datetime.date.today()))]

In [27]:
current_nk_flags['imo'].unique()

array([8985036, 8829579, 8989874, 8021579, 7036383, 9534652, 6409466,
       8898831, 7522435, 8882052, 7238785, 8898740, 8925012, 8106496,
       7827861, 7379618, 8610461, 8221222, 8122347, 8989795, 8963600,
       8712362, 8151415, 6817948, 7313822, 8987333, 7303803, 8018900,
       8217685, 8999544, 8415433, 8810384, 8518962, 9103635, 7914212,
       8731497, 9159787, 8898738, 8974099, 8890580, 8629101, 7418373,
       8922709, 8406858, 7524275, 8672691, 8922412, 8730998, 8850499,
       8882040, 8838128, 7206641, 8883707, 8408595, 8672299, 8672287,
       8732908, 8720436, 7354565, 7122120, 8990550, 8613578, 8202472,
       7722607, 8503735, 8989068, 8916293, 8824220, 8966535, 8201870,
       8123145, 8029789, 8004428, 7209095, 8891857, 7122235, 8891883,
       8661575, 8732037, 9055618, 8210699, 7916777, 7913438, 7910383,
       8850011, 7431117, 7506118, 8832227, 7327419, 7225324, 7209112,
       8150605, 8891871, 8743660, 8651415, 8680014, 8603286, 7518642,
       7402178, 9004

In [ ]:
res_date = pd.to_datetime(datetime.date.today())
baddy_list = []
imo_list = []
vio_list = ['gbo', 'doc_comp_co', 'reg_owner']

df['imo'].apply(find_2270_baddies)
baddies = pd.DataFrame(baddy_list)
baddies = baddies[baddies['ship_current_status'] == "In Service/Commission"]

In [18]:
def cleaner(x):
    x = x.replace("\xa0", " ")
    return x
un_list = pd.read_excel("~/Desktop/Assignments/GSP2/un_countries.xlsx", header = 0)
un_list = list(un_list['flag_states'].apply(cleaner))

In [19]:
def find_2321_baddies(imo):
    today = pd.to_datetime(datetime.date.today())
    global baddy_list
    global res_date
    global un_list
    global imo_list
    if imo not in imo_list:
        vessel = df2.loc[df2['imo'] == imo]
        non_nk_flags = vessel.loc[(vessel['relation'] == 'flag') & (vessel['node'].isin(un_list))]
        for index, row in non_nk_flags.iterrows():
            if (row['start_date'] <= today)  &  (row['end_date'] >= res_date):
                            baddy_dict = {}
                            baddy_dict['relation'] = row['relation']
                            baddy_dict['imo'] = row['imo']
                            baddy_dict['start_date'] = row['start_date']
                            baddy_dict['end_date'] = row['end_date']
                            baddy_dict['node'] = row['node']
                            baddy_list.append(baddy_dict)
            imo_list.append(imo)

In [23]:
res_date = pd.to_datetime("November 30, 2016", format='%B %d, %Y')
baddy_list = []
imo_list = []

df2['imo'].apply(find_2321_baddies)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
3233    None
3234    None
3235    None
3236    None
3237    None
3238    None
3239    None
3240    None
3241    None
3242    None
3243    None
3244    None
3245    None
3246    None
3247    None
3248    None
3249    None
3250    None
3251    None
3252    None
3253    None
3254    None
3255    None
3256    None
3257    None
3258    None
3259    None
3260    None
3261    None
3262    None
Name: imo, dtype: object

In [36]:
bad_flaggers = pd.DataFrame(baddy_list)
bad_flaggers["node"].value_counts()

Togo            14
Tanzania         9
Panama           8
Fiji             7
Sierra Leone     6
Unknown          6
Jamaica          5
Belize           4
Micronesia       4
Moldova          3
Palau            2
Mongolia         2
South Korea      1
Cook Islands     1
Name: node, dtype: int64

# Count Table for Visualization

In [20]:
df2['node'][df2['node'] == 'Tanzania (Zanzibar)'] = "Tanzania" 

/Users/niccdias/anaconda/envs/py3k/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
def count_table(df, first_date):
    flags = df[df['relation'] == 'flag']
    today = pd.to_datetime(datetime.date.today())
    rows = pd.date_range(first_date, today)
    columns = flags['node'].unique()
    count_df = pd.DataFrame(index=rows, columns=columns)
    count_df = count_df.fillna(0)
    for day in rows:
        for index, row in flags.iterrows():
            if (day >= row['start_date']) and (day <= row['end_date']):
                cell = count_df[row['node']][day]
                cell += 1
                count_df.set_value(day, row['node'], cell)
            else:
                continue
    return count_df

In [27]:
first_date = pd.to_datetime("February 1, 2017", format='%B %d, %Y')
count_df = count_table(df2, first_date)

In [42]:
columns = []
for column in count_df:
    cells = []
    for cell in count_df[column]:
        if int(cell) > 5:
            cells.append(1)
    if sum(cells) == 0:
        del count_df[column]

In [24]:
count_df = count_df.interpolate(method='linear')

In [28]:
count_df.to_csv(path_or_buf='~/Desktop/Assignments/GSP2/count_table.csv')